In [0]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from datetime import date

# Obtener la fecha actual en formato ISO 8601
current_date_iso = date.today().isoformat()

# Configura Spark
spark = SparkSession.builder.getOrCreate()

key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
# URL de la API (reemplaza con tu clave y parámetros)
api_url = "https://api.weatherapi.com/v1/history.json?q=Lima&dt="+current_date_iso+"&end_dt="+current_date_iso+"&key="+key

# Llama a la API
response = requests.get(api_url)
if response.status_code == 200:
    weather_data_Lima = response.json()
else:
    raise Exception(f"Error en la API: {response.status_code}")

api_url = "https://api.weatherapi.com/v1/history.json?q=Bogota&dt="+current_date_iso+"&end_dt="+current_date_iso+"&key="+key

# Llama a la API
response = requests.get(api_url)
if response.status_code == 200:
    weather_data_bogota = response.json()
else:
    raise Exception(f"Error en la API: {response.status_code}")


weather_data = [weather_data_Lima, weather_data_bogota]



flattened_data = []
for item in weather_data:
    location = item["location"]
    for forecast_day in item["forecast"]["forecastday"]:
        for hour in forecast_day["hour"]:
            flattened_data.append({
                "name": location["name"],
                "region": location["region"],
                "country": location["country"],
                "lat": location["lat"],
                "lon": location["lon"],
                "tz_id": location["tz_id"],
                "localtime_epoch": location["localtime_epoch"],
                "localtime": location["localtime"],
                "date": forecast_day["date"],
                "time_epoch": hour["time_epoch"],
                "time": hour["time"],
                "temp_c": hour["temp_c"],
                "temp_f": hour["temp_f"],
                "is_day": hour["is_day"],
                "condition_text": hour["condition"]["text"],
                "condition_icon": hour["condition"]["icon"],
                "condition_code": hour["condition"]["code"]
            })

# Crear DataFrame de PySpark

schema = StructType([

    StructField("name", StringType(), True),
    StructField("region", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True),
    StructField("tz_id", StringType(), True),
    StructField("localtime_epoch", IntegerType(), True),
    StructField("localtime", StringType(), True),

    StructField("date", StringType(), True),

    StructField("time_epoch", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("temp_c", DoubleType(), True),
    StructField("temp_f", DoubleType(), True),
    StructField("is_day", IntegerType(), True),
    StructField("condition_text", StringType(), True),
    StructField("condition_icon", StringType(), True),
    StructField("condition_code", IntegerType(), True)
])

# Crear el DataFrame
df = spark.createDataFrame(flattened_data, schema)
df.show()




table_name = "default.new_weather_data_history"  # Cambia 'default' por tu esquema si es necesario

# Escribe el DataFrame en la tabla Delta
df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(table_name)

print(f"Datos insertados correctamente en la tabla: {table_name}")



+----+------+-------+------+------+------------+---------------+----------------+----------+----------+----------------+------+------+------+--------------+--------------------+--------------+
|name|region|country|   lat|   lon|       tz_id|localtime_epoch|       localtime|      date|time_epoch|            time|temp_c|temp_f|is_day|condition_text|      condition_icon|condition_code|
+----+------+-------+------+------+------------+---------------+----------------+----------+----------+----------------+------+------+------+--------------+--------------------+--------------+
|Lima|  Lima|   Peru|-12.05|-77.05|America/Lima|     1734073870|2024-12-13 02:11|2024-12-13|1734066000|2024-12-13 00:00|  19.1|  66.3|     0| Partly cloudy|//cdn.weatherapi....|          1003|
|Lima|  Lima|   Peru|-12.05|-77.05|America/Lima|     1734073870|2024-12-13 02:11|2024-12-13|1734069600|2024-12-13 01:00|  19.0|  66.2|     0| Partly cloudy|//cdn.weatherapi....|          1003|
|Lima|  Lima|   Peru|-12.05|-77.05|